<a href="https://colab.research.google.com/github/rosie-xue/NLP/blob/main/Prompt%20Learning%20%2B%20IMDB%20Text%20Sentiment%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 提示学习（Prompt Learning）

## NLP的训练范式  

<img src="./NLP训练范式.png" alt="NLP训练范式" width="1200">

目前学术界一般将NLP任务的发展分为四个阶段，即NLP四范式：

1. **第一范式：基于传统机器学习模型的范式**
   - 例如：tf-idf 特征+朴素贝叶斯等机器算法。

2. **第二范式：基于深度学习模型的范式**
   - 例如：word2vec 特征 + LSTM 等深度学习算法。
   - 相比于第一范式，模型准确度有所提高，特征工程的工作也有所减少。

3. **第三范式：基于预训练模型 + finetuning的范式**
   - 例如：BERT + finetuning 的NLP任务。
   - 相比于第二范式，模型准确度显著提高，但是模型也随之变得更大，但小数据集就可训练出好模型。

4. **第四范式：基于预训练模型 + Prompt + 预测的范式**
   - 例如：BERT + Prompt 的范式。
   - 相比于第三范式，模型训练所需的训练数据显著减少。

<img src="./prompt示例.png" alt="prompt示例" width="1200">


<img src="./下游任务微调.png" alt="下游任务微调" width="500">


第三范式是想要预训练模型更好的应用在下游任务，需要利用下游数据对模型参数微调；首先，模型在预训练的时候，采用的训练形式：自回归、自编码，这与下游任务形式存在极大的 gap，不能完全发挥预训练模型本身的能力

必然导致：较多的数据来适应新的任务形式——>少样本学习能力差、容易过拟合。

上下游任务形式存在gap其次，现在的预训练模型参数量越来越大，为了一个特定的任务去 finetuning 一个模型，然后部署于线上业务，也会造成部署资源的极大浪费。


## 为什么提示学习？
<img src="./性能对比.png" alt="性能对比" width="500">

预训练模型中存在大量知识；预训练模型本身具有少样本学习能力。GPT-3 提出的 In-Context Learning，也有效证明了在 Zero-shot、Few-shot场景下，模型不需要任何参数，就能达到不错的效果，特别是近期很火的GPT3.5系列中的 ChatGPT。

### Prompt Learning 的本质
- **核心理念**：将所有下游任务统一成预训练任务。
- **方法**：以特定的模板，将下游任务的数据转成自然语言形式，充分挖掘预训练模型本身的能力。
- **关键步骤**：
  1. 设计预训练语言模型的任务。
  2. 设计输入模板样式（Prompt Engineering）。
  3. 设计label 样式及模型的输出映射到label 的方式（Answer Engineering）。

### Prompt Learning 的形式
- **示例：电影评论情感分类**
  - 原始输入：特效非常酷炫，我很喜欢。
  - Prompt 输入：
    1. 提示模板1: 特效非常酷炫，我很喜欢。这是一部[MASK]电影。
    2. 提示模板2: 特效非常酷炫，我很喜欢。这部电影很[MASK]。
  - **作用**：将训练数据转成自然语言的形式，并在合适的位置 MASK，以激发预训练模型的能力。

### 模板框架类别映射/Verbalizer
- **目标**：选择合适的预测词，并将这些词对应到不同的类别。
- **优势**：类别映射通过构建提示学习样本，只需要少量数据的 Prompt Tuning，就可以实现很好的效果，具有较强的零样本/少样本学习能力。


## 硬模板-PET（Pattern Exploiting Training）

<img src="./PET.png" alt="PET" width="500">

PET 是一种较为经典的提示学习方法。它将问题建模成一个完形填空问题，然后优化最终的输出词。虽然 PET 也是在优化整个模型的参数，但是相比于传统的 Finetuning 方法，对数据量需求更少。

### 建模方式
- 传统模型只需对 P(l|x) 建模（l 是 label）。
- 加入 Prompt P 及标签映射（verbalizer），问题更新为：\[P(l|x, P, M)\]（M 表示模型，s 为某个 prompt 下生成对应 word 的 logits）。
- 通过 softmax 得到概率。
- 加入 MLM loss 进行联合训练。

### 具体做法
- 在少量监督数据上，给每个 Prompt 训练一个模型。
- 对于无监督数据，集成同一样本的多个 prompt 预测结果，采用平均或加权（根据准确率分配权重）方式，归一化得到概率分布，作为无监督数据的 soft label。
- 在得到的 soft label 上 finetune 最终模型。


## 软模板- P Tuning（V1/V2）

![P-tuning-v2](./P-tuning-v2.png)

P Tuning 是一种不再依赖硬模板设计或搜索的方法。它在输入端直接插入若干可被优化的 Pseudo Prompt Tokens，自动化地寻找连续空间中的知识模板。

### 特点
- **不依赖人工设计**：完全自动化的过程。
- **参数优化**：优化的参数极少，避免了过拟合。也可全量微调，退化成传统的 finetuning。

### 工作原理
- 传统离散 prompt 直接将模板 T 的每个 token 映射为对应的 embedding。
- P-Tuning 将模板 T 中的 Pi（Pseudo Prompt）映射为一个可训练的参数 hi。
- 优化关键：将自然语言的 hard prompt 替换为可训练的 soft prompt。
- 使用双向 LSTM 对模板 T 中的 pseudo token 序列进行表征。
- 引入少量自然语言提示的锚字符（Anchor）提升效率，如“capital”。

### 具体做法
- **初始化模板**：例如 "The capital of [X] is [mask]"。
- **替换输入**：例如 [X] 处替换为输入 “Britain”，即预测 Britain 的首都。
- **挑选模板中的一个或多个 token 作为 soft prompt**。
- **处理 soft prompt**：将所有 soft prompt 送入 LSTM，获得每个 soft prompt 的隐状态向量 h。
- **模型运用**：将初始模板送入 BERT 的 Embedding Layer，所有 soft prompt 的 token embedding 用 h 代替，然后预测 mask。

### 核心结论
- 在基于全量数据的大模型中，仅微调 prompt 相关的参数，可媲美 fine-tuning 的表现。


# 思维链（Chain of Thought）

![cot](./cot.png)

## 背景知识

思维链是一种自然语言处理（NLP）技术，旨在通过模拟人类解决问题的逻辑思维过程来提升大规模语言模型的性能。这种方法特别适用于复杂问题解决任务，如数学问题、逻辑推理等。

### 基本原理
思维链方法基于一个假设：如果模型能够像人类一样“思考”并解释其解题过程，那么它解决问题的能力会更强。在实践中，这通常意味着向模型提供一个问题，并引导它生成一系列逻辑步骤，而不是直接输出最终答案。

### 发展历史
思维链的概念源于人类解决问题时通常不直接跳到答案，而是经历一系列思考步骤的观察。最近，随着大规模语言模型如GPT-3的发展，研究者开始探索如何将这种“思维过程”集成到模型中，以提高其解决复杂问题的能力。

## 应用案例

### 数学问题解答
在解决数学问题时，思维链可以用来展示逐步的计算过程。例如，对于一个简单的加法问题，模型会首先识别它需要添加两个数，然后逐步计算总和，最后给出答案。

### 逻辑推理
在逻辑推理任务中，如解释现象或推断因果关系，思维链可以帮助模型明确每个逻辑步骤，从而提供更加准确和可解释的答案。

### 语言理解
在更复杂的语言理解任务中，如阅读理解或对话理解，思维链帮助模型在给出答案之前，先阐述其理解和推理过程，这有助于提高答案的准确性和可靠性。

## 结论

思维链作为一种提高大规模语言模型解决问题能力的技术，不仅增强了模型的准确性，还提供了一种可解释AI的途径。通过模拟人类思考的步骤，思维链使得模型的输出不仅是简单的答案，而是一个完整、逻辑连贯的解决方案。


In [ ]:
!pip install datasets
!pip install zhipuai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling PyJWT-2.3.0:
      Successfully uninstalled PyJWT-2.3.0


In [ ]:
import torch
import random
from tqdm import tqdm
from transformers import BertTokenizer, BertForMaskedLM
from torch.utils.data import DataLoader
from datasets import load_dataset
from sklearn.metrics import accuracy_score

In [ ]:
#加载训练集，从训练集和验证集得到例子
dataset = load_dataset("imdb")

random_train_indices = random.sample(range(len(dataset["train"])), 1000)
random_test_indices = random.sample(range(len(dataset["test"])), 1000)

test_dataset = dataset["test"].select(random_test_indices)
train_dataset = dataset["train"].select(random_train_indices)

test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

### 基于 BERT 提示工程的 IDMB 文本情感分类

In [ ]:
def evaluate_bert_model(model, tokenizer, test_loader, device):#把mask问题放在前面
    model = model.to(device)
    model.eval()
    predictions = []
    references = []
    with torch.no_grad():
        for batch in tqdm(test_loader):
            batch_texts = batch['text']
            # TODO: 构建prompt模板（如何构建提示模板？）
            masked_texts = ["This movie was [MASK]. "+text for text in batch_texts]
            inputs = tokenizer(masked_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
            outputs = model(**inputs)

            pred = outputs.logits
            mask_token_index = torch.where(inputs.input_ids == tokenizer.mask_token_id)[1]

            # TODO: 构建标签映射词表（如何更好地构建？ good bad的概率都很低？ 引入更多标签词汇terrible nice）
            positive_id = tokenizer.convert_tokens_to_ids("good")
            negative_id = tokenizer.convert_tokens_to_ids("bad")

            # 分析每个文本的情感
            for j in range(pred.shape[0]):
                mask_pred = pred[j, mask_token_index[j], :]
                positive_prob = mask_pred[positive_id].item()
                negative_prob = mask_pred[negative_id].item()
                sentiment = 1 if positive_prob > negative_prob else 0  #如果积极概率更大，就判断为正向
                predictions.append(sentiment)
            references.extend(batch["label"].tolist())

    return accuracy_score(references, predictions)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

acc = evaluate_bert_model(model, tokenizer, test_loader, device)
print("Accuracy: ", acc)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  1%|          | 1/125 [00:42<1:27:07, 42.15s/it]


KeyboardInterrupt: 

### 基于生成式大语言模型（API）提示工程的 IDMB 文本情感分类

In [ ]:
# 如需调用不同模型 API，修改以下代码

from zhipuai import ZhipuAI

client = ZhipuAI(api_key="635bc426d639ff8105708d498f2ae68e.NDH5DUAeOuPlRsBT") # 填写您自己的APIKey
def get_api_response(text):
    response = client.chat.completions.create(
        model="glm-3-turbo",  # 填写需要调用的模型名称
        messages=[
            {"role": "user", "content": text},
        ],
    )
    return response.choices[0].message

In [ ]:
def evaluate_llm_model(test_dataset, train_dataset, topk=0):
    predictions = []
    references = test_dataset["label"]
    template = "\nWhat is the emotional preference of this movie review? (positive/negative)"

    exmaples = ''
    samples = random.sample(range(len(train_dataset)), topk)
    for i in samples:
        if train_dataset[i]["label"] == 1:
            label = "positive"
        else:
            label = "negative"
        exmaples += train_dataset[i]["text"] + template + '\n' + label + '\n'

    for text in tqdm(test_dataset["text"]):
        prompt = text + template
        response = str(get_api_response(prompt)).lower()
        sentiment = 1 if "positive" in response.lower() else 0
        predictions.append(sentiment)
    return accuracy_score(references, predictions)

In [ ]:
acc = evaluate_llm_model(test_dataset, train_dataset, topk=0)
print("Accuracy: ", acc)

  0%|          | 4/1000 [00:09<38:52,  2.34s/it]


KeyboardInterrupt: 

# QUESTIONS：

1. **Prompt Learning 与传统微调的区别是什么？**
- Prompt Learning可以在少量数据甚至无数据的情况下工作。只需调整提示或少量参数，计算成本低。可以通过修改提示快速适应不同任务，不需要重新训练模型。

2. **在设计有效的提示（prompts）时应考虑哪些因素？**
- 任务匹配度：明确任务类型和目标，分类、生成、填空等。例如，分类任务需要明确指示模型进行分类操作。
- 语言和格式：使用自然且符合人类语言习惯的提示。确保提示格式在整个数据集中保持一致，以避免混淆模型。
- 提示内容：提示应该包含足够的信息，使模型能够准确地理解并执行任务，但也不要过于冗长。使用与任务高度相关的关键词，避免模糊和泛化的提示，尽量具体和明确。
- 提示位置：将提示放在输入文本的开头，有助于模型快速理解任务。或在输入文本中嵌入提示，使其更自然地融入上下文。
- 实验和优化：可以设计多个不同的提示进行实验，观察哪种提示效果最好。根据实验结果，逐步优化和调整提示的内容和结构。
- 模型特性：不同大小和架构的模型对提示的响应可能不同，需针对特定模型设计提示。还要了解模型在预训练过程中接触的数据类型和风格。

3. **在实现基于 BERT 提示工程的情感分类任务时，选择哪些候选标签可能更有效，为什么？**
- 简单的正面、负面标签可能不够精准和细致，所以在某些情感分析任务中，可能需要更细化的情感类别。例如：高兴、愤怒、悲伤、惊讶、中性。
- 还可以根据具体的情感分析任务，选择与任务背景相关的标签。例如，在分析电影评论时：喜欢、不喜欢，在分析产品评价时：满意、不满意
- 此外，考虑到上下文和使用场景，选择能够捕捉到更细微情感变化的标签。例如，在社交媒体分析中：赞、踩、笑、怒

4. **对于提示工程，Zero-shot 和 Few-shot 的区别是什么？Few-shot如何选择恰当的例子？**

**区别**：
1. **Zero-shot（零样本）**：面对新任务，在没有训练样本的情况下，根据其已有的知识和训练数据来尝试完成该任务。例如，当使用ChatGPT时，如果不给任何示例，直接让它回答问题，ChatGPT会根据你的问题按照它自己训练的数据进行回复。
2. **Few-shot（少样本）**：给模型提供少量特定任务的训练样本，以帮助模型更好地理解任务。如：在对ChatGPT提问时给出1个或少量示例，ChatGPT会参考这些示例来解答问题。这些示例可以帮助模型更准确地理解你的意图和需求。
**Few-shot如何选择恰当的例子**：

1. **相关性**：选择的示例应与要执行的任务高度相关,并能清晰地展示任务的输入和期望的输出。
2. **多样性**：示例应具有多样性，以覆盖任务的各种可能情况。这有助于模型更全面地理解任务，并提高其泛化能力。
3. **简洁性**：示例应尽可能简洁明了，避免引入不必要的复杂性。
4. **质量**：示例的质量至关重要。应确保示例是准确的、无误的，并且能够清晰地传达任务的意图和要求。